In [1]:
#upload json file in collab

from google.colab import files
files.upload()   # Upload kaggle.json here

!mkdir -p ~/.kaggle
!cp kaggle.json ~/.kaggle/
!chmod 600 ~/.kaggle/kaggle.json
!pip install kaggle


Saving kaggle.json to kaggle.json


In [2]:
#Dataset Download (IMDB 50k Reviews)

!kaggle datasets download -d lakshmi25npathi/imdb-dataset-of-50k-movie-reviews
!unzip imdb-dataset-of-50k-movie-reviews.zip


Dataset URL: https://www.kaggle.com/datasets/lakshmi25npathi/imdb-dataset-of-50k-movie-reviews
License(s): other
  0% 0.00/25.7M [00:00<?, ?B/s]
100% 25.7M/25.7M [00:00<00:00, 1.39GB/s]
Archive:  imdb-dataset-of-50k-movie-reviews.zip
  inflating: IMDB Dataset.csv        


In [3]:
#Load Dataset

import pandas as pd

df = pd.read_csv("IMDB Dataset.csv")
print(df.head())

# sentiment ko 0/1 me convert karna
df['sentiment'] = df['sentiment'].map({'negative':0, 'positive':1})


                                              review sentiment
0  One of the other reviewers has mentioned that ...  positive
1  A wonderful little production. <br /><br />The...  positive
2  I thought this was a wonderful way to spend ti...  positive
3  Basically there's a family where a little boy ...  negative
4  Petter Mattei's "Love in the Time of Money" is...  positive


In [4]:
#Preprocessing (Text → Numbers)

from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from sklearn.model_selection import train_test_split

X = df['review'].values
y = df['sentiment'].values

max_words = 10000   # top 10k words
max_len = 200       # review ki max length

tokenizer = Tokenizer(num_words=max_words, oov_token="<OOV>")
tokenizer.fit_on_texts(X)

X_seq = tokenizer.texts_to_sequences(X)
X_pad = pad_sequences(X_seq, maxlen=max_len)

X_train, X_test, y_train, y_test = train_test_split(X_pad, y, test_size=0.2, random_state=42)


In [5]:
#Load Pre-trained GloVe Embeddings

!wget http://nlp.stanford.edu/data/glove.6B.zip
!unzip glove.6B.zip

import numpy as np

embedding_index = {}
with open("glove.6B.100d.txt", encoding="utf-8") as f:
    for line in f:
        values = line.split()
        word = values[0]
        coefs = np.asarray(values[1:], dtype='float32')
        embedding_index[word] = coefs

embedding_matrix = np.zeros((max_words, 100))
for word, i in tokenizer.word_index.items():
    if i < max_words:
        embedding_vector = embedding_index.get(word)
        if embedding_vector is not None:
            embedding_matrix[i] = embedding_vector


--2025-08-31 08:11:27--  http://nlp.stanford.edu/data/glove.6B.zip
Resolving nlp.stanford.edu (nlp.stanford.edu)... 171.64.67.140
Connecting to nlp.stanford.edu (nlp.stanford.edu)|171.64.67.140|:80... connected.
HTTP request sent, awaiting response... 302 Found
Location: https://nlp.stanford.edu/data/glove.6B.zip [following]
--2025-08-31 08:11:27--  https://nlp.stanford.edu/data/glove.6B.zip
Connecting to nlp.stanford.edu (nlp.stanford.edu)|171.64.67.140|:443... connected.
HTTP request sent, awaiting response... 301 Moved Permanently
Location: https://downloads.cs.stanford.edu/nlp/data/glove.6B.zip [following]
--2025-08-31 08:11:27--  https://downloads.cs.stanford.edu/nlp/data/glove.6B.zip
Resolving downloads.cs.stanford.edu (downloads.cs.stanford.edu)... 171.64.64.22
Connecting to downloads.cs.stanford.edu (downloads.cs.stanford.edu)|171.64.64.22|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 862182613 (822M) [application/zip]
Saving to: ‘glove.6B.zip’

glov

In [6]:
#Build LSTM Model

from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Embedding, LSTM, Dense, Dropout

model = Sequential([
    Embedding(input_dim=max_words,
              output_dim=100,
              weights=[embedding_matrix],
              input_length=max_len,
              trainable=False),
    LSTM(128, dropout=0.2, recurrent_dropout=0.2),
    Dense(1, activation='sigmoid')
])

model.compile(loss='binary_crossentropy',
              optimizer='adam',
              metrics=['accuracy'])

model.summary()


/usr/local/lib/python3.12/dist-packages/keras/src/layers/core/embedding.py:97: UserWarning: Argument `input_length` is deprecated. Just remove it.
  warnings.warn(


Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ embedding (Embedding)           │ ?                      │     1,000,000 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ lstm (LSTM)                     │ ?                      │   0 (unbuilt) │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense (Dense)                   │ ?                      │   0 (unbuilt) │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 1,000,000 (3.81 MB)

 Trainable params: 0 (0.00 B)

 Non-trainable params: 1,000,000 (3.81 MB)

In [7]:
#Train Model

history = model.fit(
    X_train, y_train,
    batch_size=64,
    epochs=5,
    validation_split=0.2
)


Epoch 1/5
500/500 ━━━━━━━━━━━━━━━━━━━━ 176s 336ms/step - accuracy: 0.6318 - loss: 0.6345 - val_accuracy: 0.7955 - val_loss: 0.4476
Epoch 2/5
500/500 ━━━━━━━━━━━━━━━━━━━━ 192s 326ms/step - accuracy: 0.7650 - loss: 0.4911 - val_accuracy: 0.8404 - val_loss: 0.3626
Epoch 3/5
500/500 ━━━━━━━━━━━━━━━━━━━━ 200s 322ms/step - accuracy: 0.8196 - loss: 0.3978 - val_accuracy: 0.8515 - val_loss: 0.3331
Epoch 4/5
500/500 ━━━━━━━━━━━━━━━━━━━━ 209s 337ms/step - accuracy: 0.8426 - loss: 0.3614 - val_accuracy: 0.8664 - val_loss: 0.3067
Epoch 5/5
500/500 ━━━━━━━━━━━━━━━━━━━━ 196s 325ms/step - accuracy: 0.8539 - loss: 0.3390 - val_accuracy: 0.8748 - val_loss: 0.2954


In [8]:
#Evaluate model

loss, accuracy = model.evaluate(X_test, y_test)
print(f"Test Accuracy: {accuracy*100:.2f}%")


313/313 ━━━━━━━━━━━━━━━━━━━━ 37s 116ms/step - accuracy: 0.8744 - loss: 0.2962
Test Accuracy: 87.17%


In [11]:
#Test on Custom Review

def predict_sentiment(review):
    seq = tokenizer.texts_to_sequences([review])
    pad = pad_sequences(seq, maxlen=max_len)
    pred = model.predict(pad)[0][0]
    return "Positive 😀" if pred > 0.5 else "Negative 😞"

print(predict_sentiment("The movie was absolutely wonderful and inspiring!"))
print(predict_sentiment("Worst movie ever, total waste of time."))


1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 122ms/step
Negative 😞
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 122ms/step
Negative 😞


In [12]:
#Save Model

model.save("sentiment_lstm.h5")
print("Model Saved Successfully ✅")

Model Saved Successfully ✅
